In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from time import time

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
def main():
    use_cuda = torch.cuda.is_available()
    print("CUDA Status:",use_cuda)

    torch.manual_seed(42)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=128, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=128, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=.05, momentum=.9)

    for epoch in range(1, 300 + 1):
        t_start = time()
        train(model, device, train_loader, optimizer, epoch)
        print("Epoch time:",time()-t_start)
        test(model, device, test_loader)

main()

CUDA Status: True
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.308141
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.692225
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.488158
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.377275
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.422145
Epoch time: 3.3385660648345947

Test set: Average loss: 0.0987, Accuracy: 9686/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.288673
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.245729
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.273821
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.453431
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.231591
Epoch time: 3.630849838256836

Test set: Average loss: 0.0828, Accuracy: 9754/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.434576
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.204794
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.344632
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.230981
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.231522
Epoch time: 3.6490464210510254

Test set: Average loss: 0.0732, 

Process Process-15:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py", line 77, in __getitem__
    img = self.transform(img)
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/campus.ncl.ac.u

Traceback (most recent call last):
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-b8542b3a61cb>", line 34, in <module>
    main()
  File "<ipython-input-8-b8542b3a61cb>", line 30, in main
    train(model, device, train_loader, optimizer, epoch)
  File "<ipython-input-5-297c7ba615be>", line 3, in train
    for batch_idx, (data, target) in enumerate(train_loader):
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/home/campus.ncl.ac.uk/b6070424/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/usr/lib/python3.6/queue.py", line 164, in get
    self.not_empty.wait()
  File "/usr/lib/python3.6/threading.py", line 295, in wait

KeyboardInterrupt: 